# Kili Tutorial: Create a project

In this tutorial, we will show **how to create a mock-project through Kili's API**, interacting directly with the database. All objects will be modified directly in the app, allowing you to check that everything has been properly executed.

Interested in wine ? Well, sure, everyone is. And you probably know that quality control is a primary concern. We don't want the nectar spoiled by some young or rotten grapes. Let's build a project to classify our grapes according to its quality : 

![grapes classification](img/create_project-labelling_interface.png)

You are just 4 steps away from your first Kili project : 
1. Understand the interface
2. Create the project and fill it
3. Invite collaborators onto the project
4. Load the first assets onto the project

Additionally, for an overview of Kili, visit the [website](https://kili-technology.com), you can also check out the Kili [documentation](https://cloud.kili-technology.com/docs), or some other recipes.

## 1. `jsonInterface` of the project

Projects details regarding all the annotating tasks are pushed in Kili in the form of python dictionaries. The format of the dictionary to be pushed depends on the type of data (text, image, audio), the machine learning task(s) the project is about (e.g. simple, multiple classification, transcription, named entity recognition, object detection, etc.) and their order.

The following cell shows you how to write this JSON for a project, and will describe every field.

In [ ]:
title = 'Grapes classification'
description = 'Identification and qualification of grapes'
input_type = 'IMAGE'
json_interface = {
    "jobRendererWidth": 0.2,
    "jobs": {
        "CLASSIFICATION_JOB": {
            "mlTask": "CLASSIFICATION",
            "content": {
                "categories": {
                    "FULLY_GROWN": {
                        "name": "Fully grown"
                    },
                    "PARTIALLY_GROWN": {
                        "name": "Partially grown"
                    },
                    "ROTTEN": {
                        "name": "Rotten"
                    }
                },
                "input": "checkbox"
            },
            "required": 1,
            "isChild": True,
            "instruction": "Quality"
        },
        "OBJECT_DETECTION_JOB": {
            "mlTask": "OBJECT_DETECTION",
            "content": {
                "categories": {
                    "RED_GRAPES": {
                        "name": "Red grapes",
                        "color": "#941100",
                        "children": [
                            "CLASSIFICATION_JOB"
                        ]
                    },
                    "WHITE_GRAPES": {
                        "name": "White grapes",
                        "color": "#73FDFF",
                        "children": [
                            "CLASSIFICATION_JOB"
                        ]
                    }
                },
                "input": "radio"
            },
            "required": 1,
            "tools": [
                "rectangle"
            ],
            "isChild": False,
            "instruction": "Color"
        }
    }
}

The different fields of the JSON interface are :
- `jobRendererWidth`: The size allowed, in proportion, to the annotation tools; the rest of the screen is filled by the asset.
- `jobs`: List of the annotations tasks, consisting in dictionnaries as well, with their unique name as key. Each job has the following attributes :
 - `mlTask`: The task, either CLASSIFICATION, TRANSCRIPTION or OBJECT DESCRIPTION for image. You can find a more comprehensive list in the doc, section Simple and intuitive interfaces.
 - `content` : A list of different choices for the task : 
  - `categories`: Category the Asset belongs to
   - `name`: Name of the category.
   - `color`: Color chosen for this category
   - `children`: Can be empty, or is a list of nested tasks, to fill-in if this category is selected.
  - `input`: Button type, radio or checkbox, depending if the classes are mutually exclusive.
 - `required`: Is this task mandatory?
 - `isChild`: Is this task nested in another?
 - `tools`: Different types of tools are available for object detection
 - `instruction`: Instruction given to the labeler
 
 The input type argument allows you to specify the type of the project's assets, between AUDIO, IMAGE, PDF, TEXT, URL, VIDEO. It will thus choose the correct representation in the database.
 
 Now that every field is cristal clear, we just have to sign-in and let the app understand what we want !

## 2. Create the project itself

You're almost set to create your projects, the last thing you need to be cautious about is authenticating properly.
You need to **update the credentials** `api_key` and `api_endpoint` before, or to have set those as global environement variables.

In [ ]:
# Authentication
import os

# !pip install kili # uncomment if you don't have kili installed already
from kili.client import Kili

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT') # If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

In [ ]:
project = kili.create_project(title=title,
                                    description=description,
                                    input_type=input_type,
                                    json_interface=json_interface)
print(f'Created project {project["id"]}')

**Well done, your project is up-and-running !**. You can take a quick look [here](https://cloud.kili-technology.com/label/projects) to see the interface ready for your tasks to begin.

## 3. Get your project running 
Before starting labelling, you might still be wondering about inviting collaborators onboard, so you don't do all the work alone.
Just remember to select the adapted role for each personn, between ADMIN, LABELER or REVIEWER.
This can be easily done just with their email address and the following snippet, for every project you have :

In [ ]:
emails = ['collaborator1@kili-technology.com',
          'collaborator2@kili-technology.com']
for email in emails:
    kili.append_to_roles(
        project_id=project['id'], user_email=email, role='ADMIN') 

## 4. At last, start the work !
You're there, you just have to import your assets now, annd you will be set. 
Those must be provided to the function `append_many_to_dataset`, that takes in argument all your data as an array, as well as the names they will have inside the app

In [ ]:
assets_to_import = ["https://www.gourmetodyssey.fr/blog/medias/Pictures/Grappe_300_225.JPG",
                    "https://www.dovepress.com/cr_data/article_submission_image/s1000" + \
                    "00/100653/magyar-soos_fig290.jpg", "https://i-reg.unimedias.fr/s" + \
                    "ites/art-de-vivre/files/styles/large/public/Import/raisin-blanc-" + \
                    "grappe-vigne_istock.jpg?auto=compress%2Cformat&crop=faces%2Cedge" + \
                    "s&cs=srgb&fit=crop","https://encrypted-tbn0.gstatic.com/images?q" + \
                    "=tbn%3AANd9GcRNGko3BcU2cu4ZJ2VjMM8cO1ktSndft2E2LnDiVkQMaZm-Ptq7&" + \
                    "usqp=CAU","https://www.syngenta.fr/sites/g/files/zhg141/f/styles" + \
                    "/syngenta_large/public/2016/07/29/grappes-raisin-stade-fermeture" + \
                    "-de-grappe-1024.jpg?itok=BoTOgbC4","https://previews.123rf.com/i" + \
                    "mages/stegarau/stegarau1708/stegarau170800038/84770765-grappes-d" + \
                    "e-raisin-dans-les-rang%C3%A9es-de-vignes-au-coucher-du-soleil.jpg"]

In [ ]:
id_assets = kili.append_many_to_dataset(
    project_id=project['id'],
    content_array=assets_to_import,
    external_id_array=['grappes' + str(k) for k in range(len(assets_to_import))])

## Summary

In this tutorial, we introduced the concept of Kili interface settings, and we've seen in detail how to create a project. If you enjoyed this tutorial, check out the other recipes for other tutorials that you may find interesting, including demonstrations of how to use Kili.

You can also visit the [website](https://kili-technology.com) or the [documentation](https://cloud.kili-technology.com/docs) for more info!

In [ ]:
assert(id_assets['id']== project['id']) #just a check to assert that everything is running